In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
export_path = './export'
file_list = { 
    'map': 'song-mapping.csv', 
    'siamzone': 'song-siamzone.csv', 
    'spotify': 'song-spotify.csv',
}

In [3]:
# , fieldnames=['id', 'song_id', 'artist', 'title']

song_list = []

with open(f'{export_path}/{file_list["spotify"]}') as f:
    songs = csv.DictReader(f, dialect=csv.unix_dialect)
    for song in songs:
        title = song['title'].split('(')[0].strip()
        title = title.split('-')[0].strip() if '-' in title else title.strip()
        title = title.split('[')[0].strip() if '[' in title else title.strip()
        
        if 'feat' in title.lower():
            title = title[:title.lower().index('feat')].strip().replace('.', '')
        
        song['mod_title'] = title
        
        song_list.append({
            'id': song['id'],
            'track_id': song['track_id'],
            'artist': song['artist'],
            'title': song['title'],
            'mod_title': title,
            'link': song['link'],
            'has_lyrics': song['has_lyrics'],
        })

In [6]:
song_list_no_lyrics_filtered = filter(lambda c: c['has_lyrics'] == 'False', song_list)
lyrics_song = list(song_list_no_lyrics_filtered)
print(len(lyrics_song))

2855


In [7]:
artist_group = {}

# id, spotify, siamzone

with open(f'{export_path}/{file_list["map"]}') as f:
    artist_mapping = csv.DictReader(f, dialect=csv.unix_dialect)
    artist_keys = set()
    
    artist_map = []
    
    for artist in artist_mapping:
        if artist['siamzone'] is not None:
            artist_keys.add(artist['spotify'])
            artist_map.append({
                'id': artist['id'],
                'spotify': artist['spotify'],
                'siamzone': artist['siamzone'],
            })

    artist_keys = list(artist_keys)
    
    for artist in artist_keys:
        artist_filter = filter(lambda c: c['spotify'] == artist, artist_map)
        artist_filter = list(artist_filter)
        if len(artist_filter):
            if artist in artist_group:
                artist_group[artist].append(k['siamzone'] for k in artist_filter)
            else:
                artist_group[artist] = [k['siamzone'] for k in artist_filter]
                

In [8]:
artist_group

{'Chart Suchart': ['ชาติ สุชาติ แซ่เห้ง'],
 '4EVE': ['โฟร์อีฟ\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(4eve)',
  'โฟร์อีฟ\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(4Eve)'],
 '60Miles': ['ซิกส์ตี้ ไมลส์',
  'ซิกส์ตี้ ไมลส์\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(60 Miles)'],
 'Rooftop, AUTTA': ['รูฟท็อป',
  'รูฟท็อป\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Rooftop)'],
 'NAP A LEAN': ['แนป อะ ลีน',
  'แนป อะ ลีน\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Nap a Lean)',
  'แนป อะ ลีน\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Nap A Lean)'],
 "Yes'sir Days, Oat Pramote": ['เยสเซอร์เดย์',
  "เยสเซอร์เดย์\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Yes'Sir Day)",
  'เยสเซอร์เดย์\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Yes\x92sir Days)',
  "เยสเซอร์เดย์\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Yes'sir Days)",
  'เยสเซอร์เดย์\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Yes\x92Sir Days)'],
 'Kong Huayrai': ['ก้อง ห้วยไร่'],
 'VANGOE': ['แวนโก๊ะ\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Vangoe)'],
 'YOUNGOHM, 1MILL': ['โอม รัธพงศ์ ภูรีสิทธิ์',
  'โอม รัธพงศ์ ภูรีสิทธิ์\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Youngohm)'],
 'Whal & Dolp

In [9]:
siamzone_group = []

# id, song_id, artist_name, title

with open(f'{export_path}/{file_list["siamzone"]}') as f:
    siamzone_list = csv.DictReader(f, dialect=csv.unix_dialect)
    for siamzone in siamzone_list:
        q = dict(siamzone)
        siamzone_group.append(dict(siamzone))

In [10]:
filtered_list = []
for s in song_list:
    s['found'] = s['artist'] in artist_group
    s['siamzone_id'] = None
    if s['found']:
        group = artist_group[s['artist']]
        k = filter(lambda c: c['artist_name'] in group and s['title'] in c['title'], siamzone_group)
        k = list(k)
        if len(k):
            s['siamzone_id'] = k[0]['id']
        
        filtered_list.append(s)

In [11]:
spotify_id = [f['id'] for f in filtered_list]

filtered_song = [s for s in song_list if s['id'] not in spotify_id and s['has_lyrics']]
print(len(filtered_list))

926


In [11]:
filtered_list

[{'id': '18',
  'track_id': '4tM3ygBuSy0QzB0ihYZQcf',
  'artist': 'SPRITE, GUYGEEGEE',
  'title': 'ทน',
  'mod_title': 'ทน',
  'link': 'https://open.spotify.com/track/4tM3ygBuSy0QzB0ihYZQcf',
  'has_lyrics': 'False',
  'found': True,
  'siamzone_id': '19474'},
 {'id': '19',
  'track_id': '0AGhwXsWpVOwjHY5yf4dtD',
  'artist': 'F.HERO, Txrbo',
  'title': 'จำเลยรัก - Defendant Of Love',
  'mod_title': 'จำเลยรัก',
  'link': 'https://open.spotify.com/track/0AGhwXsWpVOwjHY5yf4dtD',
  'has_lyrics': 'False',
  'found': True,
  'siamzone_id': None},
 {'id': '21',
  'track_id': '19CoWVqDyUbiqxoyMf4HkJ',
  'artist': 'Anatomy Rabbit',
  'title': 'Extraordinary',
  'mod_title': 'Extraordinary',
  'link': 'https://open.spotify.com/track/19CoWVqDyUbiqxoyMf4HkJ',
  'has_lyrics': 'False',
  'found': True,
  'siamzone_id': '18445'},
 {'id': '22',
  'track_id': '5sXuqzLmT1z6Gnf3S2TzSz',
  'artist': 'Da Endorphine',
  'title': 'สองใจ - เพลงประกอบละคร วันทอง',
  'mod_title': 'สองใจ',
  'link': 'https://ope

In [12]:
f_song = [f for f in filtered_list if f['has_lyrics'] == 'True' or None != f['siamzone_id']]

In [13]:
df = pd.DataFrame.from_dict(f_song)

In [14]:
df[['artist', 'title', 'mod_title', 'has_lyrics', 'siamzone_id']]

,artist,title,mod_title,has_lyrics,siamzone_id
0,"SPRITE, GUYGEEGEE",ทน,ทน,False,19474
1,Anatomy Rabbit,Extraordinary,Extraordinary,False,18445
2,Violette Wautier,กักตัว,กักตัว,False,19658
3,SERIOUS BACON,1001 (You're Lovely),1001,False,19021
4,PP Krit,"หลอกกันทั้งนั้น (Fake News) - From ""แปลรักฉันด...",หลอกกันทั้งนั้น,True,None
...,...,...,...,...,...
707,"เพชร-แตน, เอ มหาหิงค์",เจ็บละเนาะ,เจ็บละเนาะ,True,11449
708,The Others,อย่าคิดมาก,อย่าคิดมาก,True,11296
709,"UrboyTJ, Mindset",ไม่อยากฟัง,ไม่อยากฟัง,True,11065
710,Mint Pattarasaya,พบเธอก่อน,พบเธอก่อน,True,11421


In [15]:
df.to_csv('export/name_cleansing.csv', index=False)

In [16]:
k = filter(lambda c: c['id'] == '19456', siamzone_group)
print(list(k))

[{'id': '19456', 'song_id': '20880', 'artist_name': 'ไอเฮดมันเดย์\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(I Hate Monday)', 'title': 'ยิ่งใกล้ยิ่งเก็บ'}]
